In [8]:
import torch
import torch.nn as nn
from torchvision.models import densenet161
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os

In [9]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.3.0
True
NVIDIA GeForce RTX 2060
cuda:0


In [10]:
class ImageDataset(Dataset):
    def __init__(self, root,  transform, is_valid_file):
        self.root = root
        self.transform = transform
        self.is_valid_file = is_valid_file
        self.samples = self.__get_samples()

    def __len__(self):
        return len(self.samples)

    def __get_samples(self):
        ListFiles=[]
        listofFiles = os.listdir(self.root)
        
        listofFiles.sort(key=lambda x: int(x.split('.')[0]))

        for file in listofFiles:
            ListFiles.append(self.root + '/' + file)
        return ListFiles

    def loader_fc(self, path):
        img = np.load(path)
        return img

    def __getitem__(self, index: int):
        path = self.samples[index]
        sample = self.loader_fc(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample

In [11]:
class MinMaxNormalize(object):
    def __init__(self, min_val=0.0, max_val=1.0):
        self.min_val = min_val
        self.max_val = max_val

    def __call__(self, tensor):
        min_tensor = tensor.min()
        max_tensor = tensor.max()
        normalized_tensor = (tensor - min_tensor) / (max_tensor - min_tensor)
        normalized_tensor = normalized_tensor * (self.max_val - self.min_val) + self.min_val
        return normalized_tensor

data_transforms =  transforms.Compose([
        transforms.ToTensor(),
        transforms.ConvertImageDtype(torch.float32),
        MinMaxNormalize(min_val=0.0, max_val=1.0),
        transforms.Resize(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [12]:
path = "./data/test/test"

test_dataset = ImageDataset(root = path,
                    transform = data_transforms,
                    is_valid_file = True)

dataloader = DataLoader(test_dataset,
                    batch_size=1,
                    shuffle=False,
                    num_workers=0,
                    pin_memory=True)

dataset_size = len(test_dataset)

In [14]:
model = densenet161()
num_features = model.classifier.in_features

model.classifier = nn.Sequential(nn.Dropout(0.3),
                                 nn.Linear(num_features, 2))

model.to(device)
model.load_state_dict(torch.load(os.path.join(".", "models", "submit_model_params.pt")))


<All keys matched successfully>

In [16]:
model.eval()

y_pred = torch.tensor([], device=device)
with torch.inference_mode():
    for file in dataloader:
        file = file.to(device)
        
        outputs = model(file)

        _, preds = torch.max(outputs, 1)
        y_pred = torch.hstack((y_pred, preds))

y_pred = y_pred.cpu().numpy()
y_pred = y_pred.tolist()

In [17]:
test_files = [x.split('/')[-1] for x in test_dataset.samples]

preds = []
for pred in y_pred:
    if pred == 0:
        preds.append('benign')
    else:
        preds.append('malignant')

df = pd.DataFrame({'ID': test_files, 'Class': preds})

In [19]:
df.to_csv('./submission/submit_densenet161.csv', index=False)